<a href="https://colab.research.google.com/github/Leoli04/llms-notebooks/blob/main/huggingface/hf_nlp_06_Tokenizers_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tokenizers

这一章我们将主要学习以下内容：
- 如何在新文本语料库上训练与给定检查点使用的标记器类似的新标记器
- fast tokenizers的特点
- 当今 NLP 中使用的三种主要子词标记化算法之间的差异
- 如何使用 🤗 Tokenizers 库从头开始构建分词器并根据一些数据对其进行训练

### 在旧的分词器基础上训练新的分词器

这节介绍三种主要算法

> ⚠️ 训练分词器与训练模型不同！
**模型训练**使用随机梯度下降来使每个批次的损失稍微小一些。它本质上是随机的（这意味着你必须设置一些种子才能在进行两次相同的训练时获得相同的结果）。
**训练分词器**是一个统计过程，试图确定哪些子词最适合给定语料库，并且用于选择它们的确切规则取决于分词算法。它是确定性的，这意味着在同一语料库上使用相同的算法进行训练时，您总是会得到相同的结果。

#### 组装语料库

🤗 Transformers 中有一个非常简单的 API，您可以使用它来训练与现有标记器具有相同特征的新标记器： `AutoTokenizer.train_new_from_iterator()` 。下面我们以训练一个

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!apt install git-lfs

##### 加载新数据集

加载code_search_net数据集，该数据集是为 CodeSearchNet 挑战赛创建的，包含来自 GitHub 上多种编程语言的开源库的数百万个函数。这里，我们将加载该数据集的 Python 部分

In [ ]:
from datasets import load_dataset

# This can take a few minutes to load, so grab a coffee or tea while you wait!
raw_datasets = load_dataset("code_search_net", "python")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code_search_net
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


##### 查看数据集结构

In [ ]:
# 查看数据结构
raw_datasets["train"]

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 412178
})

In [ ]:
print(raw_datasets["train"][123456]["whole_func_string"])

def core_properties(self):
        """
        Instance of |CoreProperties| holding the read/write Dublin Core
        document properties for this presentation. Creates a default core
        properties part if one is not present (not common).
        """
        try:
            return self.part_related_by(RT.CORE_PROPERTIES)
        except KeyError:
            core_props = CorePropertiesPart.default()
            self.relate_to(core_props, RT.CORE_PROPERTIES)
            return core_props


##### 将数据集转换为文本列表的迭代器

好处：

- 使我们的分词器运行得更快（对批量文本进行训练，而不是逐个处理单个文本）
- 如果您的语料库很大，🤗 数据集不会将所有内容加载到 RAM 中，而是将数据集的元素存储在磁盘上。

如果你的数据集比较下，可以使用这种方式: 会将所有内容加载到内存

```
training_corpus = [raw_datasets["train"][i: i + 1000]["whole_func_string"] for i in range(0, len(raw_datasets["train"]), 1000)]
```

以下是两个推荐分批加载的方式


In [ ]:

def get_training_corpus():
    return (
        raw_datasets["train"][i : i + 1000]["whole_func_string"]
        for i in range(0, len(raw_datasets["train"]), 1000)
    )


training_corpus = get_training_corpus()

In [ ]:
def get_training_corpus():
    dataset = raw_datasets["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples["whole_func_string"]

training_corpus = get_training_corpus()

#### 训练新的分词器

加载要与模型配对的分词器（此处为 GPT-2），好处：我们就不必指定任何有关标记化算法或我们想要使用的特殊标记的信息；我们的新分词器将与 GPT-2 完全相同，唯一改变的是词汇，这将由我们的语料库上的训练决定。

##### 加载GPT-2分词器

In [ ]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

##### 查看GPT2分词python 函数的能力

In [ ]:
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

tokens = old_tokenizer.tokenize(example)
tokens

['def',
 'Ġadd',
 '_',
 'n',
 'umbers',
 '(',
 'a',
 ',',
 'Ġb',
 '):',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ"""',
 'Add',
 'Ġthe',
 'Ġtwo',
 'Ġnumbers',
 'Ġ`',
 'a',
 '`',
 'Ġand',
 'Ġ`',
 'b',
 '`',
 '."',
 '""',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġreturn',
 'Ġa',
 'Ġ+',
 'Ġb']

如上，gpt2原有的分词器有一些特殊符号，例如 Ġ 和 Ċ ，分别表示空格和换行符。但是也有一些问题：
- 将方法中原本是整体缩进分割成一个个独立的空格
- 将函数名称分割得有点奇怪，不习惯看到带有 _ 字符的单词；

##### 使用新的数据集`training_corpus`在GPT2分词器的基础上训练新的分词器

In [ ]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

##### 查看新分词器效果

In [ ]:
tokens = tokenizer.tokenize(example)
tokens

['def',
 'Ġadd',
 '_',
 'numbers',
 '(',
 'a',
 ',',
 'Ġb',
 '):',
 'ĊĠĠĠ',
 'Ġ"""',
 'Add',
 'Ġthe',
 'Ġtwo',
 'Ġnumbers',
 'Ġ`',
 'a',
 '`',
 'Ġand',
 'Ġ`',
 'b',
 '`."""',
 'ĊĠĠĠ',
 'Ġreturn',
 'Ġa',
 'Ġ+',
 'Ġb']

经过新的数据训练后，我们可以不仅看到表示空格和换行符的特殊符号 Ġ 和 Ċ ，也可以看到我们的分词器学习了一些针对 Python 函数语料库高度特定的标记：例如，有一个 ĊĠĠĠ 标记表示缩进，还有一个 Ġ""" 标记表示文档字符串开头的三个引号。分词器还正确分割 _ 上的函数名称。

In [ ]:
print(len(tokens))
print(len(old_tokenizer.tokenize(example)))

27
36


In [ ]:
example = """class LinearLayer():
    def __init__(self, input_size, output_size):
        self.weight = torch.randn(input_size, output_size)
        self.bias = torch.zeros(output_size)

    def __call__(self, x):
        return x @ self.weights + self.bias
    """
tokenizer.tokenize(example)

['class',
 'ĠLinear',
 'Layer',
 '():',
 'ĊĠĠĠ',
 'Ġdef',
 'Ġ__',
 'init',
 '__(',
 'self',
 ',',
 'Ġinput',
 '_',
 'size',
 ',',
 'Ġoutput',
 '_',
 'size',
 '):',
 'ĊĠĠĠĠĠĠĠ',
 'Ġself',
 '.',
 'weight',
 'Ġ=',
 'Ġtorch',
 '.',
 'randn',
 '(',
 'input',
 '_',
 'size',
 ',',
 'Ġoutput',
 '_',
 'size',
 ')',
 'ĊĠĠĠĠĠĠĠ',
 'Ġself',
 '.',
 'bias',
 'Ġ=',
 'Ġtorch',
 '.',
 'zeros',
 '(',
 'output',
 '_',
 'size',
 ')',
 'ĊĊĠĠĠ',
 'Ġdef',
 'Ġ__',
 'call',
 '__(',
 'self',
 ',',
 'Ġx',
 '):',
 'ĊĠĠĠĠĠĠĠ',
 'Ġreturn',
 'Ġx',
 'Ġ@',
 'Ġself',
 '.',
 'weights',
 'Ġ+',
 'Ġself',
 '.',
 'bias',
 'ĊĠĠĠĠ']

#### 保存分词器

In [ ]:
tokenizer.save_pretrained("code-search-net-tokenizer")


('code-search-net-tokenizer/tokenizer_config.json',
 'code-search-net-tokenizer/special_tokens_map.json',
 'code-search-net-tokenizer/vocab.json',
 'code-search-net-tokenizer/merges.txt',
 'code-search-net-tokenizer/added_tokens.json',
 'code-search-net-tokenizer/tokenizer.json')

In [ ]:
tokenizer.push_to_hub("code-search-net-tokenizer")

CommitInfo(commit_url='https://huggingface.co/leoli04/code-search-net-tokenizer/commit/7e23d1c7c33344c53ae17ed04000d0906ba027f8', commit_message='Upload tokenizer', commit_description='', oid='7e23d1c7c33344c53ae17ed04000d0906ba027f8', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("leoli04/code-search-net-tokenizer")

tokenizer_config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/822k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/467k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.17M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

### Fast tokenizers的特殊能力

在前面的章节，我们仅使用它们来对输入进行标记化或将 ID 解码回文本，但是tokenizers （尤其是由 🤗 Tokenizers 库支持的标记化器）可以做更多事情。在本节中，我们将仔细研究 🤗 Transformer 中标记器的功能。



在下面的讨论中，我们经常会区分“慢”和“快”标记器。慢速分词器是在 🤗 Transformers 库中用 Python 编写的，而快速版本是 🤗 分词器提供的，它是用 Rust 编写的。下面是他们的耗时对比：

| Fast tokenizer  | Slow tokenizer |         |
| --------------- | -------------- | ------- |
| `batched=True`  | 10.8s          | 4min41s |
| `batched=False` | 59.2s          | 5min3s  |

> ⚠️ 当标记单个句子时，快速版本可能会更慢！只有同时并行标记大量文本时，您才能清楚地看到差异。

#### 批量编码

分词器的输出不是一个简单的 Python 字典,而是一个特殊的 BatchEncoding 对象。它是字典的子类,但是具有 `fast tokenizers`大部分要使用的附加方法

除了并行化功能之外，快速标记器的关键功能是它们始终跟踪最终标记所来自的原始文本范围 - 我们称之为 `offset mapping`（偏移映射）的功能。

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
example = "My name is Sylvain and I work at Hugging Face in Brooklyn."
encoding = tokenizer(example)
print(type(encoding))

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

<class 'transformers.tokenization_utils_base.BatchEncoding'>


AutoTokenizer 类默认选择快速分词器，我们也可以使用 tokenizer 的 is_fast属性或 encoding 的is_fast属性来检查

In [ ]:
tokenizer.is_fast

True

In [ ]:
encoding.is_fast

True

查看上面fast tokenize，在内部都做了什么：

In [ ]:
encoding.tokens()

['[CLS]',
 'My',
 'name',
 'is',
 'S',
 '##yl',
 '##va',
 '##in',
 'and',
 'I',
 'work',
 'at',
 'Hu',
 '##gging',
 'Face',
 'in',
 'Brooklyn',
 '.',
 '[SEP]']

In [ ]:
encoding.word_ids()

[None, 0, 1, 2, 3, 3, 3, 3, 4, 5, 6, 7, 8, 8, 9, 10, 11, 12, None]

我们可以看到分词器的特殊标记 [CLS] 和 [SEP] 被映射到 None，，然后每个标记都被映射到它源自的单词（依靠 ## 前缀来实现这一点，这只适用于类似 BERT 的分词器）。

In [ ]:
start, end = encoding.word_to_chars(3)
example[start:end]

'Sylvain'

其他分词器的尝试

In [ ]:
tokenizer_ro = AutoTokenizer.from_pretrained("roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
example_1="81s"
example_2="IDs"

encoding_1_bert = tokenizer(example_1)
encoding_2_bert = tokenizer(example_2)

encoding_1_ro = tokenizer_ro(example_1)
encoding_2_ro = tokenizer_ro(example_2)

print("encoding_1_bert tokens:",encoding_1_bert.tokens())
print("encoding_2_bert tokens:",encoding_2_bert.tokens())

print("encoding_1_ro tokens:",encoding_1_ro.tokens())
print("encoding_2_ro tokens:",encoding_2_ro.tokens())

encoding_1_bert tokens: ['[CLS]', '81', '##s', '[SEP]']
encoding_2_bert tokens: ['[CLS]', 'ID', '##s', '[SEP]']
encoding_1_ro tokens: ['<s>', '81', 's', '</s>']
encoding_2_ro tokens: ['<s>', 'IDs', '</s>']


#### Inside the token-classification pipeline

 token-classification 管道中的前两个步骤与任何其他管道中的相同，但后处理稍微复杂一些。即`标记化、通过模型传递输入以及后处理`三步中的`后处理`环节有些特殊

##### 通过pipeline获取基本结果

`token-classification` 默认使用的模型是 `dbmdz/bert-large-cased-finetuned-conll03-english `；它对句子执行 NER

In [ ]:
from transformers import pipeline

token_classifier = pipeline("token-classification")
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

[{'entity': 'I-PER',
  'score': 0.99938285,
  'index': 4,
  'word': 'S',
  'start': 11,
  'end': 12},
 {'entity': 'I-PER',
  'score': 0.99815494,
  'index': 5,
  'word': '##yl',
  'start': 12,
  'end': 14},
 {'entity': 'I-PER',
  'score': 0.99590707,
  'index': 6,
  'word': '##va',
  'start': 14,
  'end': 16},
 {'entity': 'I-PER',
  'score': 0.99923277,
  'index': 7,
  'word': '##in',
  'start': 16,
  'end': 18},
 {'entity': 'I-ORG',
  'score': 0.9738931,
  'index': 12,
  'word': 'Hu',
  'start': 33,
  'end': 35},
 {'entity': 'I-ORG',
  'score': 0.976115,
  'index': 13,
  'word': '##gging',
  'start': 35,
  'end': 40},
 {'entity': 'I-ORG',
  'score': 0.9887976,
  'index': 14,
  'word': 'Face',
  'start': 41,
  'end': 45},
 {'entity': 'I-LOC',
  'score': 0.9932106,
  'index': 16,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

该模型正确地将“Sylvain”识别为一个人，将“Hugging Face”识别为一个组织，并将“Brooklyn”识别为一个位置。我们还可以要求管道将对应于同一实体的标记分组在一起：

In [ ]:
from transformers import pipeline

token_classifier = pipeline("token-classification", aggregation_strategy="simple")
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity_group': 'PER',
  'score': 0.9981694,
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.9796019,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.9932106,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

aggregation_strategy策略说明：

- "simple" 实体分数是被分词后每个token的平均值
- "first" 实体的分数是该实体的第一个token的分数（因此对于“Sylvain”，它将是 0.993828，标记 S 的分数）
- max"  实体的分数是该实体中token的最大分数
- "average" 实体的分数是组成该实体的单词分数的平均值(因此对于“Sylvain”与 "simple" 策略没有区别，但是“Hugging Face”的得分为 0.9819，“Hugging”得分的平均值为 0.975，“Face”得分为 0.98879）

##### 从输入到预测

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_checkpoint = "dbmdz/bert-large-cased-finetuned-conll03-english"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)

example = "My name is Sylvain and I work at Hugging Face in Brooklyn."
inputs = tokenizer(example, return_tensors="pt")
outputs = model(**inputs)

tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
print(inputs["input_ids"].shape)
print(outputs.logits.shape)

torch.Size([1, 19])
torch.Size([1, 19, 9])


上面结果表明，我们有一个包含19个token的序列批次，并且该模型有 9 个不同的标签，`model.config.id2label` 属性包含索引到标签的映射


In [ ]:
import torch
# 将原始输出（logits）通过 softmax 函数转换为概率分布，im=-1 参数指定了 softmax 操作应用于最后一个维度
probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)[0].tolist()
predictions = outputs.logits.argmax(dim=-1)[0].tolist()
print(predictions)

[0, 0, 0, 0, 4, 4, 4, 4, 0, 0, 0, 0, 6, 6, 6, 0, 8, 0, 0]


In [ ]:
model.config.id2label

{0: 'O',
 1: 'B-MISC',
 2: 'I-MISC',
 3: 'B-PER',
 4: 'I-PER',
 5: 'B-ORG',
 6: 'I-ORG',
 7: 'B-LOC',
 8: 'I-LOC'}

O 是不在任何命名实体中的标记的标签（它代表“外部”），然后每种类型的实体都有两个标签。标签 B-XXX 表示token位于实体 XXX 的开头，标签 I-XXX 表示token位于实体 XXX 内部。

这些 B- 和 I- 标签实际上有两种格式：IOB1 和 IOB2。 IOB2 格式（下面的粉红色）是我们引入的格式，而在 IOB1 格式（蓝色）中，以 B- 开头的标签仅用于分隔相同类型的两个相邻实体。我们使用的模型是在使用该格式的数据集上进行微调的，这就是为什么它将标签 I-PER 分配给 S 标记。

<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter6/IOB_versions.svg"/>

In [ ]:
results = []
tokens = inputs.tokens()

for idx, pred in enumerate(predictions):
    label = model.config.id2label[pred]
    if label != "O":
        results.append(
            {"entity": label, "score": probabilities[idx][pred], "word": tokens[idx]}
        )


print(results)

[{'entity': 'I-PER', 'score': 0.9993828535079956, 'word': 'S'}, {'entity': 'I-PER', 'score': 0.9981548190116882, 'word': '##yl'}, {'entity': 'I-PER', 'score': 0.995907187461853, 'word': '##va'}, {'entity': 'I-PER', 'score': 0.9992327690124512, 'word': '##in'}, {'entity': 'I-ORG', 'score': 0.9738931059837341, 'word': 'Hu'}, {'entity': 'I-ORG', 'score': 0.9761149883270264, 'word': '##gging'}, {'entity': 'I-ORG', 'score': 0.9887974858283997, 'word': 'Face'}, {'entity': 'I-LOC', 'score': 0.99321049451828, 'word': 'Brooklyn'}]


 增加偏移量：`return_offsets_mapping=True`

In [ ]:
inputs_with_offsets = tokenizer(example, return_offsets_mapping=True)
inputs_with_offsets["offset_mapping"]

[(0, 0),
 (0, 2),
 (3, 7),
 (8, 10),
 (11, 12),
 (12, 14),
 (14, 16),
 (16, 18),
 (19, 22),
 (23, 24),
 (25, 29),
 (30, 32),
 (33, 35),
 (35, 40),
 (41, 45),
 (46, 48),
 (49, 57),
 (57, 58),
 (0, 0)]

In [ ]:
example[12:14]

'yl'

In [ ]:
results = []
inputs_with_offsets = tokenizer(example, return_offsets_mapping=True)
tokens = inputs_with_offsets.tokens()
offsets = inputs_with_offsets["offset_mapping"]

for idx, pred in enumerate(predictions):
    label = model.config.id2label[pred]
    if label != "O":
        start, end = offsets[idx]
        results.append(
            {
                "entity": label,
                "score": probabilities[idx][pred],
                "word": tokens[idx],
                "start": start,
                "end": end,
            }
        )

print(results)

[{'entity': 'I-PER', 'score': 0.9993828535079956, 'word': 'S', 'start': 11, 'end': 12}, {'entity': 'I-PER', 'score': 0.9981548190116882, 'word': '##yl', 'start': 12, 'end': 14}, {'entity': 'I-PER', 'score': 0.995907187461853, 'word': '##va', 'start': 14, 'end': 16}, {'entity': 'I-PER', 'score': 0.9992327690124512, 'word': '##in', 'start': 16, 'end': 18}, {'entity': 'I-ORG', 'score': 0.9738931059837341, 'word': 'Hu', 'start': 33, 'end': 35}, {'entity': 'I-ORG', 'score': 0.9761149883270264, 'word': '##gging', 'start': 35, 'end': 40}, {'entity': 'I-ORG', 'score': 0.9887974858283997, 'word': 'Face', 'start': 41, 'end': 45}, {'entity': 'I-LOC', 'score': 0.99321049451828, 'word': 'Brooklyn', 'start': 49, 'end': 57}]


##### 对实体进行分组

把连续并用 I-XXX 标记的实体分组在一起，第一个实体除外，它可以标记为 B-XXX 或 I-XXX

In [ ]:
import numpy as np

results = []
inputs_with_offsets = tokenizer(example, return_offsets_mapping=True)
tokens = inputs_with_offsets.tokens()
offsets = inputs_with_offsets["offset_mapping"]

idx = 0
while idx < len(predictions):
    pred = predictions[idx]
    label = model.config.id2label[pred]
    if label != "O":
        # Remove the B- or I-
        label = label[2:]
        start, _ = offsets[idx]

        # Grab all the tokens labeled with I-label
        all_scores = []
        while (
            idx < len(predictions)
            and model.config.id2label[predictions[idx]] == f"I-{label}"
        ):
            all_scores.append(probabilities[idx][pred])
            _, end = offsets[idx]
            idx += 1

        # The score is the mean of all the scores of the tokens in that grouped entity
        score = np.mean(all_scores).item()
        word = example[start:end]
        results.append(
            {
                "entity_group": label,
                "score": score,
                "word": word,
                "start": start,
                "end": end,
            }
        )
    idx += 1

print(results)

[{'entity_group': 'PER', 'score': 0.998169407248497, 'word': 'Sylvain', 'start': 11, 'end': 18}, {'entity_group': 'ORG', 'score': 0.9796018600463867, 'word': 'Hugging Face', 'start': 33, 'end': 45}, {'entity_group': 'LOC', 'score': 0.99321049451828, 'word': 'Brooklyn', 'start': 49, 'end': 57}]


### QA pipeline中的Fast tokenizers


In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

#### Using the question-answering pipeline（使用问答管道）

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering")
context = """
🤗 Transformers is backed by the three most popular deep learning libraries — Jax, PyTorch, and TensorFlow — with a seamless integration
between them. It's straightforward to train your models with one before loading them for inference with the other.
"""
question = "Which deep learning libraries back 🤗 Transformers?"
question_answerer(question=question, context=context)

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

{'score': 0.98026043176651,
 'start': 78,
 'end': 106,
 'answer': 'Jax, PyTorch, and TensorFlow'}

与其他管道不同，其他管道无法截断和分割长于模型接受的最大长度的文本（因此可能会丢失文档末尾的信息），该管道可以处理非常长的上下文，并将返回回答问题

In [ ]:
long_context = """
🤗 Transformers: State of the Art NLP

🤗 Transformers provides thousands of pretrained models to perform tasks on texts such as classification, information extraction,
question answering, summarization, translation, text generation and more in over 100 languages.
Its aim is to make cutting-edge NLP easier to use for everyone.

🤗 Transformers provides APIs to quickly download and use those pretrained models on a given text, fine-tune them on your own datasets and
then share them with the community on our model hub. At the same time, each python module defining an architecture is fully standalone and
can be modified to enable quick research experiments.

Why should I use transformers?

1. Easy-to-use state-of-the-art models:
  - High performance on NLU and NLG tasks.
  - Low barrier to entry for educators and practitioners.
  - Few user-facing abstractions with just three classes to learn.
  - A unified API for using all our pretrained models.
  - Lower compute costs, smaller carbon footprint:

2. Researchers can share trained models instead of always retraining.
  - Practitioners can reduce compute time and production costs.
  - Dozens of architectures with over 10,000 pretrained models, some in more than 100 languages.

3. Choose the right framework for every part of a model's lifetime:
  - Train state-of-the-art models in 3 lines of code.
  - Move a single model between TF2.0/PyTorch frameworks at will.
  - Seamlessly pick the right framework for training, evaluation and production.

4. Easily customize a model or an example to your needs:
  - We provide examples for each architecture to reproduce the results published by its original authors.
  - Model internals are exposed as consistently as possible.
  - Model files can be used independently of the library for quick experiments.

🤗 Transformers is backed by the three most popular deep learning libraries — Jax, PyTorch and TensorFlow — with a seamless integration
between them. It's straightforward to train your models with one before loading them for inference with the other.
"""
question_answerer(question=question, context=long_context)

{'score': 0.9714871048927307,
 'start': 1892,
 'end': 1919,
 'answer': 'Jax, PyTorch and TensorFlow'}

让我们看看它是如何做到这一切的！

#### 使用模型进行问答

与任何其他管道一样，我们首先对输入进行标记，然后通过模型发送它。 question-answering 管道默认使用的检查点是 distilbert-base-cased-distilled-squad

##### 加载模型和分词器

这一步，对问题和上下文进行分词，并分词后的结果传递给模型，模型返回outputs。outputs 对象包含了模型的输出，通常包括以下属性：

- start_logits：模型预测问题答案起始位置的 logits。
- end_logits：模型预测问题答案结束位置的 logits。


In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_checkpoint = "distilbert-base-cased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
# 使用加载的分词器对问题（question）和上下文（context）进行分词，返回 PyTorch 张量
inputs = tokenizer(question, context, return_tensors="pt")
# 将分词后得到的输入传递给模型，并执行问答任务
outputs = model(**inputs)

<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter6/question_tokens.svg"/>

问答模型的工作原理与我们迄今为止看到的模型略有不同。以上图为例，模型经过训练可以预测答案开始的标记索引（此处为 21）和答案结束处的标记索引（此处为 24）。这就是为什么这些模型不返回一个 logits 张量，而是返回两个：一个用于对应于答案的开始标记的 logits，另一个用于对应于答案的结束标记的 logits。

In [ ]:
start_logits = outputs.start_logits
end_logits = outputs.end_logits
print(start_logits.shape, end_logits.shape)

torch.Size([1, 67]) torch.Size([1, 67])


屏蔽不属于上下文的索引。
我们的输入是 [CLS] question [SEP] context [SEP] ，因此我们需要屏蔽问题的标记以及 [SEP] 标记。但是，我们将保留 [CLS] 标记，因为某些模型使用它来指示答案不在上下文中。

##### 屏蔽不属于上下文的索引

In [ ]:
import torch

sequence_ids = inputs.sequence_ids()
# Mask everything apart from the tokens of the context
mask = [i != 1 for i in sequence_ids]
# Unmask the [CLS] token
mask[0] = False
mask = torch.tensor(mask)[None]

start_logits[mask] = -10000
end_logits[mask] = -10000

 使用softmax 函数，将这些 logits 转换为概率。

#####  将logits 转换为概率

In [ ]:
start_probabilities = torch.nn.functional.softmax(start_logits, dim=-1)[0]
end_probabilities = torch.nn.functional.softmax(end_logits, dim=-1)[0]

我们将计算每个可能的 start_index 和 end_index 的概率，其中 start_index <= end_index ，然后采用概率最高的元组 (start_index, end_index) 。

假设事件“答案从 start_index 开始”和“答案在 end_index 结束”是独立的，则答案从 start_index 开始并结束的概率 end_index 处是：
`start_probabilities[start_index]×end_probabilities[end_index]`

因此，要计算所有分数，我们只需计算
`start_probabilities[start_index]×end_probabilities[end_index]` 其中 start_index <= end_index 的所有乘积。


In [ ]:
# 计算所有可能
scores = start_probabilities[:, None] * end_probabilities[None, :]

In [ ]:
# torch.triu() 函数返回作为参数传递的 2D 张量的上三角部分，因此它将为我们进行掩蔽：

scores = torch.triu(scores)

##### 获取最大值索引的start_index 和 end_index

In [ ]:
scores.argmax()

tensor(1576)

In [ ]:
# 获取最大值的索引：使用取整除 // 和取模 % 运算来获取 start_index 和 end_index
max_index = scores.argmax().item()
print(max_index)
start_index = max_index // scores.shape[1]
end_index = max_index % scores.shape[1]
print(scores[start_index, end_index])

1576
tensor(0.9803, grad_fn=<SelectBackward0>)


##### 将start_index 和 end_index 转换为上下文中的字符索引

In [ ]:
inputs_with_offsets = tokenizer(question, context, return_offsets_mapping=True)
offsets = inputs_with_offsets["offset_mapping"]

start_char, _ = offsets[start_index]
_, end_char = offsets[end_index]
answer = context[start_char:end_char]

In [ ]:
result = {
    "answer": answer,
    "start": start_char,
    "end": end_char,
    "score": scores[start_index, end_index],
}
print(result)

{'answer': 'Jax, PyTorch, and TensorFlow', 'start': 78, 'end': 106, 'score': tensor(0.9803, grad_fn=<SelectBackward0>)}


In [ ]:

# torch.topk(scores, 5, dim=1).indices

tensor([[35, 36, 29, 16, 44],
        [43, 45, 44, 46, 42],
        [43, 45, 44, 46, 42],
        [43, 45, 44, 46, 42],
        [43, 45, 44, 46, 42],
        [43, 45, 44, 46, 42],
        [43, 45, 44, 46, 42],
        [43, 45, 44, 46, 42],
        [43, 45, 44, 46, 42],
        [43, 45, 44, 46, 42],
        [35, 36, 29, 16, 44],
        [35, 36, 29, 16, 44],
        [35, 36, 29, 16, 44],
        [35, 36, 29, 16, 44],
        [35, 36, 29, 16, 44],
        [35, 36, 29, 16, 44],
        [35, 36, 29, 16, 44],
        [35, 36, 29, 44, 23],
        [35, 36, 29, 44, 23],
        [35, 36, 29, 44, 23],
        [35, 36, 29, 44, 23],
        [35, 36, 29, 44, 23],
        [35, 36, 29, 44, 23],
        [35, 36, 29, 44, 23],
        [35, 36, 29, 44, 33],
        [35, 36, 29, 44, 33],
        [35, 36, 29, 44, 33],
        [35, 36, 29, 44, 33],
        [35, 36, 29, 44, 33],
        [35, 36, 29, 44, 33],
        [35, 36, 44, 33, 30],
        [35, 36, 44, 33, 32],
        [35, 36, 44, 33, 32],
        [3

#### 处理长文本

如果我们尝试对之前用作示例的问题和长上下文进行标记，我们将获得比 question-answering 管道中使用的最大长度更高的标记数量（即 384）：

In [ ]:
inputs = tokenizer(question, long_context)
print(len(inputs["input_ids"]))

461


##### 对长文本进行分割及重叠处理

truncation 如果经过分词后的句子超过了 max_length，它将被截断。

 return_overflowing_tokens 如果句子被截断，返回的将是原始的、未截断的分词结果

max_length：设置分词后序列的最大长度

stride 参数指定重叠

下面对文本内容分割、重叠验证：

In [ ]:
inputs = tokenizer(question, long_context, max_length=384, truncation="only_second")
print(tokenizer.decode(inputs["input_ids"]))

[CLS] Which deep learning libraries back [UNK] Transformers? [SEP] [UNK] Transformers : State of the Art NLP [UNK] Transformers provides thousands of pretrained models to perform tasks on texts such as classification, information extraction, question answering, summarization, translation, text generation and more in over 100 languages. Its aim is to make cutting - edge NLP easier to use for everyone. [UNK] Transformers provides APIs to quickly download and use those pretrained models on a given text, fine - tune them on your own datasets and then share them with the community on our model hub. At the same time, each python module defining an architecture is fully standalone and can be modified to enable quick research experiments. Why should I use transformers? 1. Easy - to - use state - of - the - art models : - High performance on NLU and NLG tasks. - Low barrier to entry for educators and practitioners. - Few user - facing abstractions with just three classes to learn. - A unified A

 question-answering 管道允许我们将上下文分割成更小的块，并指定最大长度。为了确保我们不会在错误的位置分割上下文以找到答案，它还包括块之间的一些重叠。

In [ ]:
# 测试文本分割和重叠
sentence = "This sentence is not too long but we are going to split it anyway."

# 每个条目最多有 6 个token并且每个条目之间有 2 个token重叠。
inputs = tokenizer(
    sentence, truncation=True, return_overflowing_tokens=True, max_length=6, stride=2
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

[CLS] This sentence is not [SEP]
[CLS] is not too long [SEP]
[CLS] too long but we [SEP]
[CLS] but we are going [SEP]
[CLS] are going to split [SEP]
[CLS] to split it anyway [SEP]
[CLS] it anyway. [SEP]


In [ ]:
print(inputs.keys())

dict_keys(['input_ids', 'attention_mask', 'overflow_to_sample_mapping'])


分词后的inputs包含：输入 ID 和注意力掩码。最后一个键 overflow_to_sample_mapping 是一个映射，它告诉我们每个结果对应于哪个句子

In [ ]:
print(inputs["overflow_to_sample_mapping"])

[0, 0, 0, 0, 0, 0, 0]


In [ ]:
sentences = [
    "This sentence is not too long but we are going to split it anyway.",
    "This sentence is shorter but will still get split.",
]
inputs = tokenizer(
    sentences, truncation=True, return_overflowing_tokens=True, max_length=6, stride=2
)

print(inputs["overflow_to_sample_mapping"])

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]


验证完后，我们开始对长文本处理：

In [ ]:
# padding="longest"：如果设置为 "longest"，则意味着较短的序列将被填充到最长序列的长度，以形成统一长度的批次。
inputs = tokenizer(
    question,
    long_context,
    stride=128,
    max_length=384,
    padding="longest",
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

##### 从 inputs 中弹出模型不需要的参数

inputs中的偏移量和 overflow_to_sample_mapping 不是模型使用的参数，因此在将其转换为张量之前，我们会将它们从 inputs 中弹出

In [ ]:
_ = inputs.pop("overflow_to_sample_mapping")
offsets = inputs.pop("offset_mapping")

inputs = inputs.convert_to_tensors("pt")
print(inputs["input_ids"].shape)

torch.Size([2, 384])


上面表示长上下文被分成两部分。然后我们将inputs传给模型

In [ ]:
outputs = model(**inputs)

start_logits = outputs.start_logits
end_logits = outputs.end_logits
print(start_logits.shape, end_logits.shape)

torch.Size([2, 384]) torch.Size([2, 384])


##### 参考短文本处理流程处理剩余步骤

In [ ]:
# 屏蔽所有填充标记（如注意力掩码所标记的）
sequence_ids = inputs.sequence_ids()
# Mask everything apart from the tokens of the context
mask = [i != 1 for i in sequence_ids]
# Unmask the [CLS] token
mask[0] = False
# Mask all the [PAD] tokens
mask = torch.logical_or(torch.tensor(mask)[None], (inputs["attention_mask"] == 0))

start_logits[mask] = -10000
end_logits[mask] = -10000

# 使用 softmax 将 logits 转换为概率
start_probabilities = torch.nn.functional.softmax(start_logits, dim=-1)
end_probabilities = torch.nn.functional.softmax(end_logits, dim=-1)

# 为所有可能的答案范围打分，然后取得分最高的范围
candidates = []
for start_probs, end_probs in zip(start_probabilities, end_probabilities):
    scores = start_probs[:, None] * end_probs[None, :]
    idx = torch.triu(scores).argmax().item()

    start_idx = idx // scores.shape[1]
    end_idx = idx % scores.shape[1]
    score = scores[start_idx, end_idx].item()
    candidates.append((start_idx, end_idx, score))

print(candidates)

# 查看每个答案的文本
for candidate, offset in zip(candidates, offsets):
    start_token, end_token, score = candidate
    start_char, _ = offset[start_token]
    _, end_char = offset[end_token]
    answer = long_context[start_char:end_char]
    result = {"answer": answer, "start": start_char, "end": end_char, "score": score}
    print(result)

[(0, 18, 0.33867061138153076), (173, 184, 0.9714869856834412)]
{'answer': '\n🤗 Transformers: State of the Art NLP', 'start': 0, 'end': 37, 'score': 0.33867061138153076}
{'answer': 'Jax, PyTorch and TensorFlow', 'start': 1892, 'end': 1919, 'score': 0.9714869856834412}


### 标准化和预标记化

以下是tokenization pipeline中步骤的概述：

<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter6/tokenization_pipeline.svg"/>

在将文本分割为子标记（根据其模型）之前，标记生成器执行两个步骤：标准化和预标记化。

#### 标准化（Normalization）

标准化这一步涉及一些常规清理，例如删除不必要的空格、小写和/或删除重音符号。

🤗 Transformers tokenizer 有一个名为 backend_tokenizer 的属性，它提供对 🤗 Tokenizers 库中底层 tokenizer 的访问：

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
print(type(tokenizer.backend_tokenizer))

<class 'tokenizers.Tokenizer'>


tokenizer 对象的 normalizer 属性有一个 normalize_str() 方法，我们可以使用它来查看规范化是如何执行的：

In [ ]:
print(tokenizer.backend_tokenizer.normalizer.normalize_str("Héllò hôw are ü?"))

hello how are u?


在此示例中，由于我们选择了 bert-base-uncased 检查点，因此规范化应用了小写并删除了重音符号。

#### 预标记（Pre-tokenization）

要了解快速分词器如何执行预分词，我们可以使用 tokenizer 对象的 pre_tokenizer 属性的 pre_tokenize_str() 方法：

In [ ]:
tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str("Hello, how are  you?")


[('Hello', (0, 5)),
 (',', (5, 6)),
 ('how', (7, 10)),
 ('are', (11, 14)),
 ('you', (16, 19)),
 ('?', (19, 20))]

我们使用的是 BERT 分词器，预分词涉及到空格和标点符号的分割。其他分词器对此步骤可以有不同的规则。例如，如果我们使用 GPT-2 分词器：

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str("Hello, how are  you?")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

[('Hello', (0, 5)),
 (',', (5, 6)),
 ('Ġhow', (6, 10)),
 ('Ġare', (10, 14)),
 ('Ġ', (14, 15)),
 ('Ġyou', (15, 19)),
 ('?', (19, 20))]

In [ ]:
# 基于 SentencePiece 算法的 T5 分词器
tokenizer = AutoTokenizer.from_pretrained("t5-small")
tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str("Hello, how are  you?")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

[('▁Hello,', (0, 6)),
 ('▁how', (7, 10)),
 ('▁are', (11, 14)),
 ('▁you?', (16, 20))]

#### SentencePiece

SentencePiece 是一种用于文本预处理的标记化算法。它将文本视为 Unicode 字符序列，并用特殊字符 ▁ 替换空格。与 Unigram 算法结合使用（参见第 7 节），它甚至不需要预标记化步骤，这对于不使用空格字符的语言（如中文或日语）非常有用。

SentencePiece 的另一个主要功能是可逆标记化：由于没有对空格进行特殊处理，因此只需将它们连接起来并用空格替换 _ 即可对标记进行解码 - 这会产生标准化文本。正如我们之前看到的，BERT 分词器删除了重复空格，因此其分词是不可逆的。

#### 算法预览

在下面的部分中，我们将深入研究三种主要的子词标记化算法：BPE（由 GPT-2 等使用）、WordPiece（例如由 BERT 使用）和 Unigram（由 T5 等使用）。

| Model         | BPE                                      | WordPiece                                                    | Unigram                                                      |
| ------------- | ---------------------------------------- | ------------------------------------------------------------ | ------------------------------------------------------------ |
| Training      | 从小词汇开始，学习合并标记的规则         | 从小词汇开始，学习合并标记的规则                             | 从大词汇量开始，学习删除标记的规则                           |
| Training step | 合并最常见对对应的标记                   | 根据该对的频率合并与具有最佳分数的该对相对应的标记，并为每个单独标记频率较低的对赋予特权 | 删除词汇表中的所有标记，从而最大限度地减少整个语料库计算的损失 |
| Learns        | 合并规则和词汇表                         | 只是词汇                                                     | 每个标记都有一个分数的词汇表                                 |
| Encoding      | 将单词拆分为字符并应用训练期间学到的合并 | 找到词汇表中从开头开始最长的子词，然后对单词的其余部分执行相同的操作 | 使用训练期间学到的分数找到最有可能的标记分割                 |

### BPE 标记（字节对编码标记）

Byte-Pair Encoding (BPE)最初是作为一种压缩文本的算法而开发的，然后由 OpenAI 在预训练 GPT 模型时用于标记化。许多 Transformer 模型都使用它，包括 GPT、GPT-2、RoBERTa、BART 和 DeBERTa。

>  💡本节深入介绍 BPE，甚至展示完整的实现。如果您只想大致了解标记化算法，可以跳到最后。

#### 训练算法

BPE 训练首先计算语料库中使用的唯一单词集（在标准化和预标记化步骤完成之后），然后通过使用用于书写这些单词的所有符号来构建词汇表。

假设我们的语料库使用这五个单词：
```
"hug", "pug", "pun", "bun", "hugs"
```
基本词汇表将为 ["b", "g", "h", "n", "p", "s", "u"] 。

如果您要标记的示例使用的字符不在训练语料库中，则该字符将转换为未知标记。这就是为什么许多 NLP 模型在分析带有表情符号的内容方面表现不佳的原因之一。

> GPT-2 和 RoBERTa 分词器有一种巧妙的方法来处理这个问题：它们不将单词用 Unicode 字符编写的，而是用字节编写的。这样，基本词汇表的大小就很小（256），但您能想到的每个字符仍然会被包含在内，并且最终不会被转换为未知标记。这个技巧称为byte-level BPE。

获得这个基本词汇后，我们添加新的标记，直到通过学习合并达到所需的词汇大小，合并是将现有词汇的两个元素合并为一个新词汇的规则。因此，一开始这些合并将创建具有两个字符的标记，然后随着训练的进行，创建更长的子词。

我们假设这些单词有以下频率：

```
("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)
```

表示 "hug" 在语料库中出现了10次， "pug" 出现了5次， "pun" 出现了12次， "bun" 出现了4次， "hugs" 5次。我们通过将每个单词分割成字符（构成我们初始词汇的字符）来开始训练，这样我们就可以将每个单词视为标记列表：
```
("h" "u" "g", 10), ("p" "u" "g", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "u" "g" "s", 5)
```

然后我们组合元素成对。由于("u", "g")出现的频率最高，有20次。

分词器学习到的第一个合并规则是 ("u", "g") -> "ug" ：
```
Vocabulary: ["b", "g", "h", "n", "p", "s", "u", "ug"]
Corpus: ("h" "ug", 10), ("p" "ug", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "ug" "s", 5)
```

接着继续组合频率最高的对 ("u", "n")（在语料库中出现了 16 次）：
```
Vocabulary: ["b", "g", "h", "n", "p", "s", "u", "ug", "un"]
Corpus: ("h" "ug", 10), ("p" "ug", 5), ("p" "un", 12), ("b" "un", 4), ("h" "ug" "s", 5)
```
然后最常见的对是 ("h", "ug")
```
Vocabulary: ["b", "g", "h", "n", "p", "s", "u", "ug", "un", "hug"]
Corpus: ("hug", 10), ("p" "ug", 5), ("p" "un", 12), ("b" "un", 4), ("hug" "s", 5)
```
按照这种方式一直继续，直到达到所需的词汇量。

#### 标记化算法

训练完以后，我们就可以开始标记化。通过应用以下步骤对新输入进行标记化：
- 标准化
- 预标记化
- 将单词拆分为单个字符
- 将学到的合并规则按顺序应用到这些拆分上

让我们以训练期间使用的示例为例，学习了三个合并规则：
```
("u", "g") -> "ug"
("u", "n") -> "un"
("h", "ug") -> "hug"
```
单词 "bug" 将被标记为 ["b", "ug"] 。但是， "mug" 将被标记为 ["[UNK]", "ug"] ，因为字母 "m" 不在基本词汇表中。

#### BPE实现

现在让我们看一下 BPE 算法的实现。

##### 准备语料库

In [83]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

##### 将语料库预先标记为单词

由于我们正在复制 BPE 标记生成器（如 GPT-2），因此我们将使用 gpt2 标记生成器进行预标记化：

In [84]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

##### 计算语料库中每个单词的频率

In [85]:
from collections import defaultdict

word_freqs = defaultdict(int)

for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

print(word_freqs)

defaultdict(<class 'int'>, {'This': 3, 'Ġis': 2, 'Ġthe': 1, 'ĠHugging': 1, 'ĠFace': 1, 'ĠCourse': 1, '.': 4, 'Ġchapter': 1, 'Ġabout': 1, 'Ġtokenization': 1, 'Ġsection': 1, 'Ġshows': 1, 'Ġseveral': 1, 'Ġtokenizer': 1, 'Ġalgorithms': 1, 'Hopefully': 1, ',': 1, 'Ġyou': 1, 'Ġwill': 1, 'Ġbe': 1, 'Ġable': 1, 'Ġto': 1, 'Ġunderstand': 1, 'Ġhow': 1, 'Ġthey': 1, 'Ġare': 1, 'Ġtrained': 1, 'Ġand': 1, 'Ġgenerate': 1, 'Ġtokens': 1})


##### 计算语料库中使用的所有字符组成的基本词汇表

In [86]:
alphabet = []

for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

print(alphabet)

[',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'Ġ']


##### 在该词汇表的开头添加模型使用的特殊标记

In [87]:
vocab = ["<|endoftext|>"] + alphabet.copy()

##### 将每个单词拆分为单独的字符

In [88]:
splits = {word: [c for c in word] for word in word_freqs.keys()}

##### 编写一个函数来计算每对的频率

In [89]:
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

In [ ]:
pair_freqs = compute_pair_freqs(splits)

for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

('T', 'h'): 3
('h', 'i'): 3
('i', 's'): 5
('Ġ', 'i'): 2
('Ġ', 't'): 7
('t', 'h'): 3


##### 找到最频繁的对

In [90]:
best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

('Ġ', 't') 7


In [94]:
merges = {("Ġ", "t"): "Ġt"}
vocab.append("Ġt")

##### 合并函数

In [91]:
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue

        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

In [92]:
splits = merge_pair("Ġ", "t", splits)
print(splits["Ġtrained"])

['Ġt', 'r', 'a', 'i', 'n', 'e', 'd']


##### 处理所有我们想要的合并

In [95]:
vocab_size = 50

while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

In [96]:
print(merges)

{('Ġ', 't'): 'Ġt', ('e', 'r'): 'er', ('Ġ', 'a'): 'Ġa', ('Ġt', 'o'): 'Ġto', ('e', 'n'): 'en', ('T', 'h'): 'Th', ('Th', 'is'): 'This', ('o', 'u'): 'ou', ('s', 'e'): 'se', ('Ġto', 'k'): 'Ġtok', ('Ġtok', 'en'): 'Ġtoken', ('n', 'd'): 'nd', ('Ġ', 'is'): 'Ġis', ('Ġt', 'h'): 'Ġth', ('Ġth', 'e'): 'Ġthe', ('i', 'n'): 'in', ('Ġa', 'b'): 'Ġab', ('Ġtoken', 'i'): 'Ġtokeni', ('Ġtokeni', 'z'): 'Ġtokeniz'}


In [97]:
print(vocab)

['<|endoftext|>', ',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'Ġ', 'Ġt', 'er', 'Ġa', 'Ġto', 'en', 'Th', 'This', 'ou', 'se', 'Ġtok', 'Ġtoken', 'nd', 'Ġis', 'Ġth', 'Ġthe', 'in', 'Ġab', 'Ġtokeni', 'Ġtokeniz']


> 💡 在同一语料库上使用 train_new_from_iterator() 不会产生完全相同的词汇。这是因为，当选择最频繁的对时，我们选择了遇到的第一个，而🤗 Tokenizers 库根据其内部 ID 选择第一个。

In [98]:
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    splits = [[l for l in word] for word in pre_tokenized_text]
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    return sum(splits, [])

In [99]:
tokenize("This is not a token.")

['Th', 'i', 's', 'Ġ', 'i', 's', 'Ġ', 'n', 'o', 't', 'Ġa', 'Ġtoken', '.']

### WordPiece 标记

WordPiece 是 Google 开发的用于预训练 BERT 的标记化算法。此后，它已在相当多基于 BERT 的 Transformer 模型中得到重用，例如 DistilBERT、MobileBERT、Funnel Transformers 和 MPNET。它在训练方面与 BPE 非常相似，但实际的标记化却有所不同。

#### 训练算法

> ⚠️ Google 从未开源其 WordPiece 训练算法的实现，因此以下是我们根据已发表文献的最佳猜测。它可能不是 100% 准确。[链接文字](https://)

WordPiece 从一个小词汇表开始，包括模型使用的特殊标记和初始字母表。

由于它通过添加前缀来识别子词（例如 BERT 的 ## ），因此每个单词最初都是通过将该前缀添加到单词内的所有字符来分割的。例如， "word" 被分割如下：
```
w ##o ##r ##d
```
因此，初始字母表包含单词开头出现的所有字符以及 WordPiece 前缀前面的单词内部出现的字符。

然后，像 BPE 一样，WordPiece 学习合并规则。主要区别在于选择要合并的对的方式。 WordPiece 使用以下公式计算每对的分数，而不是选择最频繁的对：
```
score=(freq_of_pair)/(freq_of_first_element×freq_of_second_element)
```

通过将对的频率除以其每个部分的频率的乘积，该算法会优先考虑词汇表中各个部分出现频率较低的对的合并。例如，即使 ("un", "##able") 在词汇表中出现非常频繁，它也不一定会合并 ("un", "##able")，因为 "un" 和 "##able" 两对很可能都会出现

相比之下，像 ("hu", "##gging") 这样的对可能会更快地合并，因为 "hu" 和 "##gging" 很可能单独减少频率。

我们使用 BPE 训练示例中使用的相同词汇：
```
("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)
```
这里的分割将是：

```
("h" "##u" "##g", 10), ("p" "##u" "##g", 5), ("p" "##u" "##n", 12), ("b" "##u" "##n", 4), ("h" "##u" "##g" "##s", 5)
```

所以初始词汇表将是 ["b", "h", "p", "##g", "##n", "##s", "##u"]

最频繁的配对是 ("##u", "##g") （出现了20次），但是 "##u" 的个体频率非常高，因此它的得分不是最高的（为1 / 36）。所有带有 "##u" 的配对实际上都具有相同的分数 (1 / 36)，因此最好的分数属于 ("##g", "##s") 配对 。学习到的第一个合并是 ("##g", "##s") -> ("##gs")

```
Vocabulary: ["b", "h", "p", "##g", "##n", "##s", "##u", "##gs"]
Corpus: ("h" "##u" "##g", 10), ("p" "##u" "##g", 5), ("p" "##u" "##n", 12), ("b" "##u" "##n", 4), ("h" "##u" "##gs", 5)
```

此时， "##u" 位于所有可能的对中，因此它们最终都会得到相同的分数。假设在这种情况下，第一对被合并，因此 ("h", "##u") -> "hu" 。
```
Vocabulary: ["b", "h", "p", "##g", "##n", "##s", "##u", "##gs", "hu"]
Corpus: ("hu" "##g", 10), ("p" "##u" "##g", 5), ("p" "##u" "##n", 12), ("b" "##u" "##n", 4), ("hu" "##gs", 5)

```

然后，下一个最佳得分由 ("hu", "##g") 和 ("hu", "##gs") 共享（与所有其他对的 1/21 相比，为 1/15），因此得分最高的第一对是合并：
```
Vocabulary: ["b", "h", "p", "##g", "##n", "##s", "##u", "##gs", "hu", "hug"]
Corpus: ("hug", 10), ("p" "##u" "##g", 5), ("p" "##u" "##n", 12), ("b" "##u" "##n", 4), ("hu" "##gs", 5)

```

#### 标记化算法

WordPiece 和 BPE 中的标记化不同，WordPiece 只保存最终的词汇表，而不保存学习到的合并规则。从要标记化的单词开始，WordPiece 找到词汇表中最长的子词，然后对其进行分割。例如，如果我们使用在上面的示例中学习的词汇表，对于单词 "hugs" ，词汇表中从头开始最长的子词是 "hug" ，所以我们在那里进行拆分获取 ["hug", "##s"] 。然后我们继续处理词汇表中的 "##s" ，因此 "hugs" 的标记化为 ["hug", "##s"] 。

使用 BPE，我们将应用按顺序学习的合并并将其标记为 ["hu", "##gs"] ，因此编码是不同的。

让我们看看如何对单词 "bugs" 进行标记。 "b" 是词汇表中从单词开头开始的最长子词，因此我们将其拆分并得到 ["b", "##ugs"] 。那么 "##u" 是词汇表中从 "##ugs" 开头开始的最长子词，因此我们在那里拆分并得到 ["b", "##u, "##gs"] 。最后， "##gs" 在词汇表中，因此最后一个列表是 "bugs" 的标记化。

当标记化达到无法在词汇表中找到子词的阶段时，整个单词将被标记为未知 - 因此，例如， "mug" 将被标记为 ["[UNK]"] ，就像 "bum" （即使我们可以从 "b" 和 "##u" 开始， "##m" 不是词汇表，并且生成的标记化将只是 ["[UNK]"] ，而不是 ["b", "##u", "[UNK]"] ）。这是与 BPE 的另一个区别，BPE 只会将不在词汇表中的单个字符分类为未知字符。

#### WordPiece实现

##### 准备语料库

In [100]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

##### 将语料库预先标记为单词

由于我们正在复制 WordPiece 分词器（如 BERT），因此我们将使用 bert-base-cased 分词器进行预分词：

In [102]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

##### 计算语料库中每个单词的频率

In [103]:
from collections import defaultdict

word_freqs = defaultdict(int)
for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

word_freqs

defaultdict(int,
            {'This': 3,
             'is': 2,
             'the': 1,
             'Hugging': 1,
             'Face': 1,
             'Course': 1,
             '.': 4,
             'chapter': 1,
             'about': 1,
             'tokenization': 1,
             'section': 1,
             'shows': 1,
             'several': 1,
             'tokenizer': 1,
             'algorithms': 1,
             'Hopefully': 1,
             ',': 1,
             'you': 1,
             'will': 1,
             'be': 1,
             'able': 1,
             'to': 1,
             'understand': 1,
             'how': 1,
             'they': 1,
             'are': 1,
             'trained': 1,
             'and': 1,
             'generate': 1,
             'tokens': 1})

##### 获取词汇表
字母表是由单词的所有第一个字母以及以 ## 为前缀的单词中出现的所有其他字母组成的唯一集合：

In [104]:
alphabet = []
for word in word_freqs.keys():
    if word[0] not in alphabet:
        alphabet.append(word[0])
    for letter in word[1:]:
        if f"##{letter}" not in alphabet:
            alphabet.append(f"##{letter}")

alphabet.sort()
alphabet

print(alphabet)

['##a', '##b', '##c', '##d', '##e', '##f', '##g', '##h', '##i', '##k', '##l', '##m', '##n', '##o', '##p', '##r', '##s', '##t', '##u', '##v', '##w', '##y', '##z', ',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'g', 'h', 'i', 's', 't', 'u', 'w', 'y']


##### 在该词汇表的开头添加模型使用的特殊标记

对于 BERT，它是列表 ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"] ：

In [105]:
vocab = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"] + alphabet.copy()

In [107]:
# 将 word_freqs.keys()中的每个单词拆分为单个字符，并在除第一个字符之外的其他字符前添加前缀 ##。
splits = {
    word: [c if i == 0 else f"##{c}" for i, c in enumerate(word)]
    for word in word_freqs.keys()
}

##### 计算每对的得分

In [106]:
def compute_pair_scores(splits):
    letter_freqs = defaultdict(int)
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            letter_freqs[split[0]] += freq
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            letter_freqs[split[i]] += freq
            pair_freqs[pair] += freq
        letter_freqs[split[-1]] += freq

    scores = {
        pair: freq / (letter_freqs[pair[0]] * letter_freqs[pair[1]])
        for pair, freq in pair_freqs.items()
    }
    return scores

In [108]:
pair_scores = compute_pair_scores(splits)
for i, key in enumerate(pair_scores.keys()):
    print(f"{key}: {pair_scores[key]}")
    if i >= 5:
        break

('T', '##h'): 0.125
('##h', '##i'): 0.03409090909090909
('##i', '##s'): 0.02727272727272727
('i', '##s'): 0.1
('t', '##h'): 0.03571428571428571
('##h', '##e'): 0.011904761904761904


In [109]:
# 得分最高的一对
best_pair = ""
max_score = None
for pair, score in pair_scores.items():
    if max_score is None or max_score < score:
        best_pair = pair
        max_score = score

print(best_pair, max_score)

('a', '##b') 0.2


In [110]:
# 因此，要学习的第一个合并是 ('a', '##b') -> 'ab'
vocab.append("ab")

In [111]:
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue
        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                merge = a + b[2:] if b.startswith("##") else a + b
                split = split[:i] + [merge] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

In [112]:
splits = merge_pair("a", "##b", splits)
splits["about"]

['ab', '##o', '##u', '##t']

In [113]:
vocab_size = 70
while len(vocab) < vocab_size:
    scores = compute_pair_scores(splits)
    best_pair, max_score = "", None
    for pair, score in scores.items():
        if max_score is None or max_score < score:
            best_pair = pair
            max_score = score
    splits = merge_pair(*best_pair, splits)
    new_token = (
        best_pair[0] + best_pair[1][2:]
        if best_pair[1].startswith("##")
        else best_pair[0] + best_pair[1]
    )
    vocab.append(new_token)

In [114]:
# 查看生成的词汇表
print(vocab)

['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]', '##a', '##b', '##c', '##d', '##e', '##f', '##g', '##h', '##i', '##k', '##l', '##m', '##n', '##o', '##p', '##r', '##s', '##t', '##u', '##v', '##w', '##y', '##z', ',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'g', 'h', 'i', 's', 't', 'u', 'w', 'y', 'ab', '##fu', 'Fa', 'Fac', '##ct', '##ful', '##full', '##fully', 'Th', 'ch', '##hm', 'cha', 'chap', 'chapt', '##thm', 'Hu', 'Hug', 'Hugg', 'sh', 'th', 'is', '##thms', '##za', '##zat', '##ut']


我们从第一个单词的开头开始查找最大的子单词并将其拆分，然后我们在第二部分重复该过程，依此类推，对该单词的其余部分以及文本中的以下单词进行：

In [116]:
def encode_word(word):
    tokens = []
    while len(word) > 0:
        i = len(word)
        while i > 0 and word[:i] not in vocab:
            i -= 1
        if i == 0:
            return ["[UNK]"]
        tokens.append(word[:i])
        word = word[i:]
        if len(word) > 0:
            word = f"##{word}"
    return tokens

In [117]:
print(encode_word("Hugging"))
print(encode_word("HOgging"))

['Hugg', '##i', '##n', '##g']
['[UNK]']


In [118]:
# 对文本进行标记的函数
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    encoded_words = [encode_word(word) for word in pre_tokenized_text]
    return sum(encoded_words, [])

In [119]:
tokenize("This is the Hugging Face course!")

['Th',
 '##i',
 '##s',
 'is',
 'th',
 '##e',
 'Hugg',
 '##i',
 '##n',
 '##g',
 'Fac',
 '##e',
 'c',
 '##o',
 '##u',
 '##r',
 '##s',
 '##e',
 '[UNK]']

### Unigram 分词

Unigram 算法经常在 SentencePiece 中使用，这是 AlBERT、T5、mBART、Big Bird 和 XLNet 等模型使用的标记化算法。

#### 训练算法
与 BPE 和 WordPiece 相比，Unigram 的工作方向相反：它从一个大词汇表开始，然后从中删除标记，直到达到所需的词汇表大小。有多种选项可用于构建基本词汇表：例如，我们可以采用预标记单词中最常见的子字符串，或者在词汇量较大的初始语料库上应用 BPE。

在训练的每一步，Unigram 算法都会计算给定当前词汇量的语料库的损失。然后，对于词汇表中的每个符号，算法计算如果删除该符号，总体损失会增加多少，并寻找增加最少的符号。这些符号对语料库整体损失的影响较小，因此从某种意义上说，它们“不太需要”，并且是删除的最佳候选者。

```
("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)
```
对于这个例子，我们将采用所有严格的子字符串作为初始词汇：

```
["h", "u", "g", "hu", "ug", "p", "pu", "n", "un", "b", "bu", "s", "hug", "gs", "ugs"]
```

#### 标记化算法

以下是词汇表中所有可能的子词的频率：

```
("h", 15) ("u", 36) ("g", 20) ("hu", 15) ("ug", 20) ("p", 17) ("pu", 17) ("n", 16)
("un", 16) ("b", 4) ("bu", 4) ("s", 5) ("hug", 15) ("gs", 5) ("ugs", 5)
```

所有频率之和为 210，子词 "ug" 的概率为 20/210。

具体看[这里](https://huggingface.co/learn/nlp-course/chapter6/7?fw=pt)

### 构件分词器



🤗 Tokenizers 库的构建是为了为每个步骤提供多个选项，您可以将它们混合和匹配在一起。在本节中，我们将了解如何从头开始构建分词器，然后，您将能够构建您能想到的任何类型的分词器！

分词器标记化包括几个步骤：

- 规范化（任何认为必要的文本清理，例如删除空格或重音符号、Unicode 规范化等）
- 预标记化（将输入分割为单词）
- 通过模型运行输入（使用预先标记化的单词来生成标记序列）
- 后处理（添加分词器的特殊标记，生成注意掩码和标记类型 ID）

整个过程的演示：

<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter6/tokenization_pipeline.svg"/>

更准确地说，该库是围绕 Tokenizer 类构建的，其中的构建块在子模块中重新分组：

- normalizers 包含您可以使用的所有可能的 Normalizer 类型(完整列表请参见[此处](https://huggingface.co/docs/tokenizers/api/normalizers))
- pre_tokenizers 包含您可以使用的所有可能的 PreTokenizer 类型（完整列表请参见[此处](https://huggingface.co/docs/tokenizers/api/pre-tokenizers)）
- models 包含您可以使用的各种类型的 Model ，例如 BPE 、 WordPiece 和 Unigram （完整[在此列出](https://huggingface.co/docs/tokenizers/api/models)）
- trainers 包含可用于在语料库上训练模型的所有不同类型的 Trainer （每种模型一个；完整列表请参见[此处](https://huggingface.co/docs/tokenizers/api/trainers)）。
- post_processors 包含您可以使用的各种类型的 PostProcessor （完整列表请参见[此处](https://huggingface.co/docs/tokenizers/api/post-processors)）。
- decoders 包含可用于解码标记化输出的各种类型的 Decoder （[此处](https://huggingface.co/docs/tokenizers/components#decoders)为完整列表）。

您可以在[此处](https://huggingface.co/docs/tokenizers/components)找到构建块的完整列表。

#### 获取语​​料库

In [121]:
from datasets import load_dataset

dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="train")


def get_training_corpus():
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["text"]

🤗 分词器也可以直接在文本文件上进行训练。

In [ ]:
with open("wikitext-2.txt", "w", encoding="utf-8") as f:
    for i in range(len(dataset)):
        f.write(dataset[i]["text"] + "\n")

#### 从头开始构建 WordPiece 分词器

要使用 🤗 Tokenizers 库构建分词器，我们首先使用 model 实例化 Tokenizer 对象，然后设置其 normalizer 、 pre_tokenizer 、 post_processor 和 decoder 属性为我们想要的值。

对于此示例，我们将使用 WordPiece 模型创建一个 Tokenizer ：

##### 创建Tokenizer

In [122]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

我们必须指定 unk_token ，以便模型知道在遇到以前从未见过的字符时返回什么。

##### 标准化

标记化的第一步是标准化，所以让我们从标准化开始。由于 BERT 被广泛使用，所以有一个 BertNormalizer ，其中有我们可以为 BERT 设置的经典选项： lowercase 和 strip_accents ，clean_text 删除所有控制字符并将重复空格替换为单个空格；和 handle_chinese_chars ，在汉字周围放置空格。要复制 bert-base-uncased 分词器，我们只需设置此标准化器即可：

In [124]:
# 复制 bert-base-uncased 分词器
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)

In [125]:
# 构建新的分词器，可以使用 Sequence 组合多个规范化器
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
)

In [126]:
# 使用 normalizer 的 normalize_str() 方法来检查它对给定文本的效果
print(tokenizer.normalizer.normalize_str("Héllò hôw are ü?"))

hello how are u?


##### 预标记化

In [127]:
# 我们可以使用预构建的 BertPreTokenizer
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()

In [128]:
# 我们可以从头开始构建它
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()


Whitespace 预分词器按空格和所有非字母、数字或下划线字符的字符进行拆分，因此从技术上讲，它按空格和标点符号进行拆分：

In [129]:
tokenizer.pre_tokenizer.pre_tokenize_str("Let's test my pre-tokenizer.")

[('Let', (0, 3)),
 ("'", (3, 4)),
 ('s', (4, 5)),
 ('test', (6, 10)),
 ('my', (11, 13)),
 ('pre', (14, 17)),
 ('-', (17, 18)),
 ('tokenizer', (18, 27)),
 ('.', (27, 28))]

如果您只想按空格分割，则应使用 WhitespaceSplit 预分词器：

In [130]:
pre_tokenizer = pre_tokenizers.WhitespaceSplit()
pre_tokenizer.pre_tokenize_str("Let's test my pre-tokenizer.")

[("Let's", (0, 5)),
 ('test', (6, 10)),
 ('my', (11, 13)),
 ('pre-tokenizer.', (14, 28))]

与规范化步骤一样，您可以使用 Sequence 来组成多个预标记器：

In [131]:
pre_tokenizer = pre_tokenizers.Sequence(
    [pre_tokenizers.WhitespaceSplit(), pre_tokenizers.Punctuation()]
)
pre_tokenizer.pre_tokenize_str("Let's test my pre-tokenizer.")

[('Let', (0, 3)),
 ("'", (3, 4)),
 ('s', (4, 5)),
 ('test', (6, 10)),
 ('my', (11, 13)),
 ('pre', (14, 17)),
 ('-', (17, 18)),
 ('tokenizer', (18, 27)),
 ('.', (27, 28))]

##### 通过模型运行输入

我们已经在初始化中指定了模型，但我们仍然需要训练它，这将需要 WordPieceTrainer 。在 🤗 Tokenizers 中实例化训练器时要记住的主要事情是，您需要将您打算使用的所有特殊标记传递给它 - 否则它不会将它们添加到词汇表中，因为它们不在训练语料库中：

In [132]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)

除了指定 vocab_size 和 special_tokens 之外，我们还可以设置 min_frequency （标记必须出现在词汇表中的次数）或更改 continuing_subword_prefix

##### 训练分词模型

In [133]:
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)


In [ ]:
# 们还可以使用文本文件来训练我们的分词器
tokenizer.model = models.WordPiece(unk_token="[UNK]")
tokenizer.train(["wikitext-2.txt"], trainer=trainer)

In [134]:
# 通过调用 encode() 方法来测试文本上的分词器
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)

['let', "'", 's', 'test', 'this', 'tok', '##eni', '##zer', '.']


获得的 encoding 是一个 Encoding ，它在其各种属性中包含标记器的所有必需输出： ids 、 type_ids 、 tokens、 offsets 、 attention_mask 、 special_tokens_mask 和 overflowing 。

##### 后处理

我们需要在开头添加 [CLS] 标记，在末尾添加 [SEP] 标记（或者在每个句子之后，如果我们有一对句子）。为此，我们将使用 TemplateProcessor ，但首先我们需要知道词汇表中 [CLS] 和 [SEP] 标记的 ID：

In [135]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

2 3


编写 TemplateProcessor 的模板，我们必须指定如何处理单个句子和一对句子。对于两者，我们编写我们想要使用的特殊标记；

第一个句子由 $A 表示，而第二个句子由 $B 表示。对于其中的每一个（特殊标记和句子），我们还在冒号后指定相应的标记类型 ID。

In [136]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],
)

In [137]:
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)

['[CLS]', 'let', "'", 's', 'test', 'this', 'tok', '##eni', '##zer', '.', '[SEP]']


In [138]:
encoding = tokenizer.encode("Let's test this tokenizer...", "on a pair of sentences.")
print(encoding.tokens)
print(encoding.type_ids)

['[CLS]', 'let', "'", 's', 'test', 'this', 'tok', '##eni', '##zer', '...', '[SEP]', 'on', 'a', 'pair', 'of', 'sentences', '.', '[SEP]']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]


##### 解码器

In [139]:
tokenizer.decoder = decoders.WordPiece(prefix="##")

In [140]:
tokenizer.decode(encoding.ids)

"let ' s test this tokenizer... on a pair of sentences."

##### 保存标记生成器

In [142]:
tokenizer.save("tokenizer.json")

##### 加载自己的标记生成器

In [143]:
new_tokenizer = Tokenizer.from_file("tokenizer.json")

##### 包装自己分词器

要在 🤗 Transformers 中使用此分词器，我们必须将其包装在 PreTrainedTokenizerFast 中。我们可以使用通用类，或者，如果我们的分词器对应于现有模型，则使用该类（此处为 BertTokenizerFast ）。如果您应用本课程来构建一个全新的分词器，则必须使用第一个选项。

关键是我们必须手动设置所有特殊标记，因为该类无法从 tokenizer 对象推断哪个标记是掩码标记，即 [CLS] 标记

In [ ]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    # tokenizer_file="tokenizer.json", # You can load from the tokenizer file, alternatively
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)

果您使用特定的标记生成器类（例如 BertTokenizerFast ），则只需指定与默认标记不同的特殊标记（此处为无）：

In [ ]:
from transformers import BertTokenizerFast

wrapped_tokenizer = BertTokenizerFast(tokenizer_object=tokenizer)

#### 从头开始构建 BPE 分词器

现在让我们构建一个 GPT-2 分词器。与 BERT 分词器一样，我们首先使用 BPE 模型初始化 Tokenizer。

与 BERT 一样，如果我们有词汇表的话，我们可以使用词汇表来初始化该模型（在这种情况下，我们需要传递 vocab 和 merges ），但由于我们将从从头开始，我们不需要这样做。我们也不需要指定 unk_token ，因为 GPT-2 使用字节级 BPE，这不需要它。

In [148]:
# 构建一个 GPT-2 分词器
tokenizer = Tokenizer(models.BPE())
# 预标记化，GPT-2 不使用标准化器，所以没有标准化这一步
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
# 输入、训练
trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

# 在文本上训练
# tokenizer.model = models.BPE()
# tokenizer.train(["wikitext-2.txt"], trainer=trainer)

# 后处理，trim_offsets = False 选项向后处理器指示我们应该保留以 'Ġ' 开头的标记的偏移量
tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
# 添加一个字节级解码器
tokenizer.decoder = decoders.ByteLevel()

In [145]:
tokenizer.pre_tokenizer.pre_tokenize_str("Let's test pre-tokenization!")


[('Let', (0, 3)),
 ("'s", (3, 5)),
 ('Ġtest', (5, 10)),
 ('Ġpre', (10, 14)),
 ('-', (14, 15)),
 ('tokenization', (15, 27)),
 ('!', (27, 28))]

In [147]:
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)

['L', 'et', "'", 's', 'Ġtest', 'Ġthis', 'Ġto', 'ken', 'izer', '.']


In [149]:
sentence = "Let's test this tokenizer."
encoding = tokenizer.encode(sentence)
start, end = encoding.offsets[4]
sentence[start:end]

' test'

In [150]:
tokenizer.decoder = decoders.ByteLevel()

In [151]:
tokenizer.decode(encoding.ids)

"Let's test this tokenizer."

保存分词器，如果我们想在 🤗 Transformers 中使用它，请将其包装在 PreTrainedTokenizerFast 或 GPT2TokenizerFast 中

In [152]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token="<|endoftext|>",
    eos_token="<|endoftext|>",
)

In [ ]:
from transformers import GPT2TokenizerFast

wrapped_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer)

#### 从头开始构建 Unigram 分词器

现在让我们构建一个 XLNet 分词器。与之前的分词器一样，我们首先使用 Unigram 模型初始化 Tokenizer ：

In [ ]:
# 创建tokenizer
tokenizer = Tokenizer(models.Unigram())

In [ ]:
from tokenizers import Regex
# 标准化
tokenizer.normalizer = normalizers.Sequence(
    [
        normalizers.Replace("``", '"'),
        normalizers.Replace("''", '"'),
        normalizers.NFKD(),
        normalizers.StripAccents(),
        normalizers.Replace(Regex(" {2,}"), " "),
    ]
)

In [ ]:
# 用于任何 SentencePiece 标记生成器的预标记生成器是 Metaspace ：
tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()

In [153]:
tokenizer.pre_tokenizer.pre_tokenize_str("Let's test the pre-tokenizer!")


[('Let', (0, 3)),
 ("'s", (3, 5)),
 ('Ġtest', (5, 10)),
 ('Ġthe', (10, 14)),
 ('Ġpre', (14, 18)),
 ('-', (18, 19)),
 ('tokenizer', (19, 28)),
 ('!', (28, 29))]

In [ ]:
# 训练模型
special_tokens = ["<cls>", "<sep>", "<unk>", "<pad>", "<mask>", "<s>", "</s>"]
trainer = trainers.UnigramTrainer(
    vocab_size=25000, special_tokens=special_tokens, unk_token="<unk>"
)
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

对于 UnigramTrainer 来说，一个非常重要的论点是 unk_token 。我们还可以传递特定于 Unigram 算法的其他参数，例如用于删除标记的每个步骤的 shrinking_factor （默认为 0.75）或用于指定最大长度的 max_piece_length 给定的标记（默认为 16）。

In [ ]:
# 在文本文件上进行训练
tokenizer.model = models.Unigram()
tokenizer.train(["wikitext-2.txt"], trainer=trainer)

In [154]:
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)

['L', 'et', "'", 's', 'Ġtest', 'Ġthis', 'Ġto', 'ken', 'izer', '.']


In [ ]:
cls_token_id = tokenizer.token_to_id("<cls>")
sep_token_id = tokenizer.token_to_id("<sep>")
print(cls_token_id, sep_token_id)

In [ ]:
# 后处理
tokenizer.post_processor = processors.TemplateProcessing(
    single="$A:0 <sep>:0 <cls>:2",
    pair="$A:0 <sep>:0 $B:1 <sep>:1 <cls>:2",
    special_tokens=[("<sep>", sep_token_id), ("<cls>", cls_token_id)],
)

In [ ]:
# 解码器
tokenizer.decoder = decoders.Metaspace()

如果我们想在 🤗 Transformer 中使用它，请将其包装在 PreTrainedTokenizerFast 或 XLNetTokenizerFast 中。使用 PreTrainedTokenizerFast 时需要注意的一件事是，在特殊标记之上，我们需要告诉 🤗 Transformers 库在左侧填充：

In [ ]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token="<s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    cls_token="<cls>",
    sep_token="<sep>",
    mask_token="<mask>",
    padding_side="left",
)

In [ ]:
# 或者
from transformers import XLNetTokenizerFast

wrapped_tokenizer = XLNetTokenizerFast(tokenizer_object=tokenizer)